## Equity Research with Multi-Agent Debate

This project draws inspiration from the multi-agent debating mechanism introduced in *AlphaAgents: Large Language Model-Based Multi-Agents for Equity Portfolio Construction (Zhao et al., 2025).*
We adapted core principles from portfolio construction to single-stock equity research. To better capture the characteristics of an individual stock rather than a diversified portfolio, we refined the approach to emphasize:

### 1. Agent Specialization
Each agent represents a distinct analytical discipline:

**Technical Agent** – evaluates returns, volatility, and momentum \
**Fundamental Agent** – interprets financial statement metrics and balance-sheet health \
**Sentiment Agent** – synthesizes news flow and analyst commentary


### 2. Structured Multi-Agent Debate
We implemented a two-round debate loop:

**Round 1**: Each agent issues an independent BUY/SELL recommendation \
**Round 2**: Each agent reviews peer opinions and may reinforce or revise its stance \


### 3. Tool-Aware Agents with Enforced Reasoning
Each agent can call role-specific tools:

**Technical** → price and momentum calculator \
**Fundamental** → fundamentals_web_tool metrics \
**Sentiment** → news_fetch_web_tool

Agents must submit opinions via save_opinion_tool using a standardized JSON schema, ensuring consistent aggregation and debate.

### 4. Debate Logging and Meeting Minutes
We introduced a **Meeting Minutes Agent** to capture all intermediate outputs and produce structured minutes summarizing the final consensus.
These minutes provide a human-readable summary of the debate—similar to an investment committee memo.

Together, these components create a compact yet realistic simulation of how multiple analysts collaborate to evaluate a stock.

## Imports and Global Setup

This cell loads required libraries, initializes memory structures for storing agent context and debate logs, and defines utility functions used throughout the system.

In [3]:
import os, json, math, re
from typing import Any, Dict, List, Optional
from datetime import timedelta, date, datetime
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import io
import contextlib
import json
from typing import Any, Dict
import requests
import logging
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
print("✅ Gemini API key setup complete.")

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)
session_service = InMemorySessionService()

CONTEXT: Dict[str, Any] = {}
OPINION_LOG: Dict[str, Dict[str, List[Dict[str, Any]]]] = {}
DEBATE_LOG: Dict[str, List[Dict[str, Any]]] = {}

✅ Gemini API key setup complete.


### We take Tesla as the target stock, defining the 30-day lookback window, risk-free rate, and unique analysis ID used throughout the multi-agent debate.

In [4]:
TICKER = "TSLA"
COMPANY = "Tesla"            
LOOKBACK_DAYS = 30          
RF_ANNUAL = 0.04                 
ANALYSIS_ID = "tsla_equity_agent"

## Agent Tools and Helper Functions

Here we define all custom tools: 

**save_context_tool** \
**get_context_tool** \
**save_opinion_tool** \
as well as utilities for computing debate snapshots

These tools enable agents to store and retrieve information.

In [6]:
def save_context_tool(analysis_id: str, ctx: Dict[str, Any]) -> dict:
    CONTEXT[analysis_id] = ctx
    return {"status":"success","analysis_id":analysis_id}

def get_context_tool(analysis_id: str) -> dict:
    c = CONTEXT.get(analysis_id)
    if not c:
        return {"status":"error","error_message":f"No context for {analysis_id}"}
    return {"status":"success","context":c}

def save_opinion_tool(analysis_id: str, agent_name: str, payload: Any) -> dict:
    aid = str(agent_name).strip().lower()

    if isinstance(payload, str):
        try:
            payload = json.loads(payload)
        except Exception:
            payload = {"rationale": payload}
    if not isinstance(payload, dict):
        payload = {"_raw": str(payload)}

    p = dict(payload)

    sent = str(p.get("sentiment","")).lower()
    if p.get("vote") not in ("BUY", "SELL"):
        if sent.startswith("bull") or sent == "positive":
            p["vote"] = "BUY"
        elif sent.startswith("bear") or sent == "negative":
            p["vote"] = "SELL"
        else:
            p["vote"] = "SELL"          # safe default
            p.setdefault("score", -0.05)
            extra = " [Auto-normalized: no explicit vote; treated as low-confidence SELL.]"
            if "rationale" in p and p["rationale"]:
                p["rationale"] += extra
            else:
                p["rationale"] = extra

    if aid in ("technical", "fundamental") and "metrics" not in p:
        p["metrics"] = {"error": "metrics missing in model output"}

    if "round" not in p or p["round"] is None:
        prev = last_opinion(analysis_id, aid)
        p["round"] = 1 if prev is None else int(prev.get("round", 1)) + 1
    else:
        try:
            p["round"] = int(p["round"])
        except Exception:
            p["round"] = 1

    p.setdefault("agent", aid)
    p.setdefault("score", 0.0)
    p.setdefault("rationale", "No rationale provided")

    OPINION_LOG.setdefault(analysis_id, {}).setdefault(aid, []).append(p)
    DEBATE_LOG.setdefault(analysis_id, []).append(
        {"agent": aid, "round": p["round"], "view": p}
    )

    print("[DEBUG save_opinion_tool] SUCCESS for", aid, "round", p["round"])
    return {"status": "success"}


def get_opinions_tool(analysis_id: str) -> dict:
    return {"status":"success","opinions":OPINION_LOG.get(analysis_id, {})}

def run_agent(agent: LlmAgent, payload: Dict[str, Any]):
    return InMemoryRunner(agent).run_debug(json.dumps(payload, indent=2))

def peer_snapshot(analysis_id: str, exclude: str) -> List[Dict[str, Any]]:
    """Latest view from each *other* agent for debate round 2."""
    out=[]
    for a, hist in OPINION_LOG.get(analysis_id, {}).items():
        if a == exclude or not hist: continue
        last = hist[-1]
        out.append({
            "agent": a,
            "round": last.get("round"),
            "vote": last.get("vote"),
            "score": last.get("score"),
            "rationale": (last.get("rationale","")[:600])
        })
    return out

def last_opinion(analysis_id: str, agent_name: str) -> Optional[Dict[str, Any]]:
    hist = OPINION_LOG.get(analysis_id, {}).get(agent_name, [])
    return hist[-1] if hist else None

def _first_nonempty(*dfs):
    for df in dfs:
        try:
            if df is not None and hasattr(df, "empty") and not df.empty:
                df.index = [str(i) for i in df.index]
                return df
        except Exception:
            pass
    return None

def _latest(df: Optional[pd.DataFrame], keys: List[str]) -> Optional[float]:
    if df is None or df.empty: return None
    low = {str(i).strip().lower(): i for i in df.index}
    for k in keys:
        if k.lower() in low:
            row = df.loc[low[k.lower()]]
            try: row = row.dropna()
            except Exception: pass
            vals = getattr(row, "values", [])
            if len(vals) > 0 and pd.notna(vals[0]):
                try: return float(vals[0])
                except Exception: continue
    return None

## Market Data & Research Tools

This cell defines three core data tools that the analyst agents call during the debate:

**prices_web_metrics_tool**

Pulls recent daily prices and volumes from yfinance and computes return-based metrics used by the Technical Agent including:

cumulative and annualized return

annualized volatility

Sharpe ratio (using a configurable annual risk-free rate)

5-day and 21-day momentum

basic volume stats (average volume, 90th percentile)

**fundamentals_web_tool**

Fetches the latest income statement, balance sheet, and cash flow data from yfinance.

Normalizes multiple possible field names (aliases) and derives key fundamental ratios for the Fundamental Agent:

gross, operating, net and free-cash-flow margins

debt-to-equity ratio

net-income-positive flag

**news_fetch_web_tool**

Gathers recent news headlines and summaries for the ticker, primarily from yf.Ticker(ticker).news, with a fallback to Yahoo Finance’s search API.

Filters by lookback_days, trims to max_items, and returns a list of dated articles with titles, publishers, and bodies.

This serves as the primary information feed for the Sentiment Agent to assess news tone and catalysts.

In [8]:
def prices_web_metrics_tool(ticker: str, lookback_days: int = 30, rf_annual: float = 0.04):

    df = yf.download(ticker, period=f"{lookback_days}d",
                     interval="1d", auto_adjust=True, progress=False, threads=True)

    if df is None or df.empty:
        return {"status": "error", "error_message": "empty dataframe from yfinance"}

    close_col = "Close" if "Close" in df.columns else ("Adj Close" if "Adj Close" in df.columns else None)
    if close_col is None:
        return {"status": "error", "error_message": "no close price column found"}

    closes = np.asarray(df[close_col].astype(float)).reshape(-1)
    if closes.size < 5:
        return {"status": "error", "error_message": "insufficient observations"}

    # Daily returns
    rets = np.diff(closes) / closes[:-1]
    n = rets.size
    rcum = float(closes[-1] / closes[0] - 1.0)

    # Annualized return and volatility
    ann_ret = float((1.0 + rcum)**(252.0 / n) - 1.0)
    ann_vol = float(np.std(rets) * math.sqrt(252.0))

    # Sharpe ratio
    rf_daily = rf_annual / 252.0
    sharpe = float(((rets - rf_daily).mean()) / (np.std(rets) + 1e-9) * math.sqrt(252.0))

    # Momentum for 5d and 21d
    m5 = float(np.prod(1.0 + rets[-min(5, n):]) - 1.0)
    m21 = float(np.prod(1.0 + rets[-min(21, n):]) - 1.0)

    # Volume stats
    vol_stats = {}
    if "Volume" in df.columns:
        vol = np.asarray(df["Volume"].astype(float)).reshape(-1)
        vol_stats = {
            "avg_daily_volume": float(vol.mean()),
            "p90_volume": float(np.quantile(vol, 0.9))
        }

    end_dt = df.index.max()
    start_dt = df.index.min()
    end_str = end_dt.date().isoformat() if hasattr(end_dt, "date") else str(end_dt)
    start_str = start_dt.date().isoformat() if hasattr(start_dt, "date") else str(start_dt)

    return {
        "status": "success",
        "period": {"start": start_str, "end": end_str},
        "metrics": {
            "cumulative_return": rcum,
            "annualized_return": ann_ret,
            "annualized_volatility": ann_vol,
            "sharpe": sharpe,
            "momentum_5d": m5,
            "momentum_21d": m21,
            "n_obs": int(n)
        },
        "volume_stats": vol_stats,
        "source": "yfinance"
    }

   

def fundamentals_web_tool(ticker: str):

    ALIASES = {
    "revenue": ["Total Revenue","Revenue","TotalRevenue"],
    "gross_profit": ["Gross Profit","GrossProfit"],
    "operating_income": ["Operating Income","Operating Income (Loss)","OperatingIncome","Operating Income or Loss"],
    "net_income": ["Net Income","NetIncome","Net Income Common Stockholders"],
    "ocf": ["Operating Cash Flow","Total Cash From Operating Activities","Net Cash Provided by Operating Activities"],
    "capex": ["Capital Expenditures","Capital Expenditure","Investments In Property Plant And Equipment"],
    "total_debt": ["Total Debt","Short Long Term Debt Total","Total Debt Net","Long Term Debt","Short Long Term Debt"],
    "equity": ["Total Stockholder Equity","Total Stockholders Equity","Total Shareholder Equity","Total Equity", "Stockholders Equity"]

    }

    try:
        t = yf.Ticker(ticker)
        inc = _first_nonempty(getattr(t, "quarterly_financials", None),
                              getattr(t, "quarterly_income_stmt", None),
                              getattr(t, "financials", None),
                              getattr(t, "income_stmt", None))
        bal = _first_nonempty(getattr(t, "quarterly_balance_sheet", None),
                              getattr(t, "balance_sheet", None))
        cfs = _first_nonempty(getattr(t, "quarterly_cashflow", None),
                              getattr(t, "cashflow", None))

        rev0 = _latest(inc, ALIASES["revenue"])
        gp0  = _latest(inc, ALIASES["gross_profit"])
        op0  = _latest(inc, ALIASES["operating_income"])
        ni0  = _latest(inc, ALIASES["net_income"])
        ocf0 = _latest(cfs, ALIASES["ocf"])
        cap0 = _latest(cfs, ALIASES["capex"])
        deb0 = _latest(bal, ALIASES["total_debt"])
        eq0  = _latest(bal, ALIASES["equity"])

        def div(a,b): 
            try: return float(a)/float(b) if (a is not None and b not in (None,0)) else None
            except Exception: return None

        fcf = (ocf0 + (cap0 or 0.0)) if ocf0 is not None else None
        metrics = {
            "gross_margin": div(gp0, rev0),
            "operating_margin": div(op0, rev0),
            "net_margin": div(ni0, rev0),
            "fcf_margin": div(fcf, rev0),
            "debt_to_equity": div(deb0, eq0),
            "net_income_positive": (ni0 is not None and ni0 > 0)
        }
        return {"status":"success","metrics":metrics,"source":"yfinance"}
    except Exception as e:
        return {"status":"error","error_message":f"{type(e).__name__}: {e}"}

def news_fetch_web_tool(ticker: str, lookback_days: int = 30, max_items: int = 100):
    
    try:        
        
        items = []
        try:
            raw = getattr(yf.Ticker(ticker), "news", None) or []
            cutoff = datetime.utcnow() - timedelta(days=int(lookback_days))
            for n in raw:
                ts = n['content']['pubDate']
                ts = pd.Timestamp(ts) if ts else datetime.utcnow()
                if ts < pd.Timestamp(cutoff).tz_localize('UTC'): 
                    continue
                items.append({
                    "date": ts.strftime("%Y-%m-%d"),
                    "title": n['content']['title'],
                    "body": n['content']['summary']
                })
        except Exception:
            pass

        if not items:
            url = "https://query2.finance.yahoo.com/v1/finance/search"
            params = {"q": ticker, "newsCount": max_items, "quotesCount": 0}
            r = requests.get(url, params=params, timeout=10)
            if r.ok:
                j = r.json()
                for n in (j.get("news") or []):
                    dt_ms = n.get("providerPublishTime") or n.get("published_at") or 0
                    ts = datetime.utcfromtimestamp(int(dt_ms)) if isinstance(dt_ms,(int,float)) else datetime.utcnow()
                    items.append({
                        "date": ts.strftime("%Y-%m-%d"),
                        "title": n.get("title",""),
                        "publisher": (n.get("publisher") or n.get("publisher_name") or ""),
                        "body": (n.get("summary") or "")
                    })

        return {"status":"success","items": items[:max_items], "source":"yfinance/yahoo"} if items else \
               {"status":"error","error_message":"no recent headlines"}
    except Exception as e:
        return {"status":"error","error_message":f"{type(e).__name__}: {e}"}

## LLM Agents for Multi-Agent Equity Debate

This cell defines the five LLM-based agents that drive the entire debate workflow:

**Intake Agent**

Lightweight setup agent.

Normalizes the user’s raw input into a structured context block: analysis_id, ticker, company, lookback_days, rf_annual.

Immediately calls save_context_tool so that all downstream agents can retrieve the same shared context.

**Technical Agent**

Acts as a technical equity analyst.

Calls prices_web_metrics_tool to compute return, volatility, Sharpe ratio, and short/medium-term momentum plus volume stats.

Issues a structured BUY/SELL opinion via save_opinion_tool with a numeric score and rationale.

**Fundamental Agent**

Plays the role of a fundamental analyst.

Calls fundamentals_web_tool to retrieve margins, leverage, and profitability indicators (e.g., gross margin, FCF margin, debt-to-equity).

Bases its BUY/SELL decision solely on these tool-derived metrics and logs its opinion via save_opinion_tool.

**Sentiment Agent**

Specializes in news and analyst sentiment.

Must call news_fetch_web_tool to pull recent headlines over the specified lookback window.

Synthesizes sentiment into a single BUY/SELL view with a score and a small sample of representative headlines, then calls save_opinion_tool exactly once.

**In Round 2, all agents review their peers’ opinions and evaluate whether to adjust their own recommendations accordingly**

**Meeting Minutes Scribe**

Doesn’t participate in the debate directly; instead, it summarizes it. It takes the final context, full debate_log, and final_decision as input, and then produces compact, professional meeting minutes in Markdown with a fixed structure: summary, agent views, key metrics, risks/watchpoints, and recommended action.

This turns the multi-agent debate into a human-readable investment committee note.

Together, these agents implement a structured, role-based multi-agent equity debate: intake sets context, three specialist analysts debate with tool-grounded evidence (and peer feedback in Round 2), and the minutes agent generates an interpretable summary of the final consensus.

In [9]:
intake_agent = LlmAgent(
    name="intake",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
Return ONLY:
{
  "context": {
    "analysis_id": "...",
    "ticker": "...",
    "company": "...",
    "lookback_days": int,
    "rf_annual": float
  }
}
Then CALL save_context_tool(analysis_id, context). Do not write any other text.
""",
    tools=[save_context_tool],
)

tech_agent = LlmAgent(
    name="technical",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=f"""

You are the Technical Agent.
As a technical equity analyst, your primary responsibility is to analyze the market trends of a given equity over a specified time horizon.

Instruction:
1. Use `prices_web_metrics_tool` to compute the following metrics for the equity
   - annualized_return
   - annualized_volatility
   - Sharpe ratio (risk-free rate provided)
   - momentum_5d
   - momentum_21d
   - volume stats
2. Analyze these metrics to identify trends and patterns. You may apply MACD theory, Chanlun principles, or other technical frameworks.
3. Provide an opinion on the likely future trend of the equity.

PEER VIEWS (Round 2 only)
If the input JSON includes "peer_views", you MUST:

1. Read every peer opinion.
2. Summarize peer arguments internally.
3. Decide whether to maintain or revise your own view.
4. In your rationale, explicitly mention:
   - Which peer opinions you agreed or disagreed with,
   - Why they strengthened or weakened your conviction.

Do NOT copy their text; generate your own synthesis.

If the tool returns an error or empty metrics, you MUST STILL ISSUE a BUY/SELL with low confidence, citing the data gap.
Input JSON: analysis_id, ticker, rf_annual, round, lookback_days, peer_views(optional).

Return ONLY JSON and CALL save_opinion_tool(analysis_id,"technical",payload):
{{
  "agent": "technical",
  "ticker": "{TICKER}",
  "round": 1|2,
  "vote": "BUY" | "SELL",
  "score": -1.0..1.0,
  "rationale": "3–6 sentences grounded in the computed metrics or the data gap",
  "metrics": {{... or {{"error":"..."}} }}
}}
""",
    tools=[get_context_tool, prices_web_metrics_tool, save_opinion_tool],
)


fund_agent = LlmAgent(
    name="fundamental",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=f"""
You are the Fundamental Agent. 
As a fundamental equity equity analyst your primary responsibility is to analyze the most recent financial statements of the company. 

Instructions:
1. Use `fundamentals_web_tool` to obtain objective metrics:
   - gross_margin
   - operating_margin
   - net_margin
   - fcf_margin
   - debt_to_equity
   - net_income_positive
2. Base your analysis solely on the information retrieved using this tool.
3. Apply interpretation:
   - Summarize what these metrics indicate about profitability, efficiency, leverage, and financial health.
   - Critique any weaknesses or risks.
   - Refine your view into a clear fundamental outlook (bullish/bearish, buy/sell).
4. You MUST call 'save_opinion_tool' EXACTLY ONCE with:
   {{
     "analysis_id": analysis_id_from_input,
     "agent_name": "fundamental",
     "payload": payload
   }}

PEER VIEWS (Round 2 only)
If the input JSON includes "peer_views", you MUST:

1. Read every peer opinion.
2. Summarize peer arguments internally.
3. Decide whether to maintain or revise your own view.
4. In your rationale, explicitly mention:
   - Which peer opinions you agreed or disagreed with,
   - Why they strengthened or weakened your conviction.

Do NOT copy their text; generate your own synthesis.

If the tool returns an error or empty metrics, you MUST STILL ISSUE a BUY/SELL with low confidence, citing the data gap.
Input JSON: analysis_id, ticker, company, round, peer_views(optional).

Return ONLY JSON and CALL save_opinion_tool(analysis_id,"fundamental",payload):
{{
  "agent": "fundamental",
  "ticker": "{TICKER}",
  "round": 1|2,
  "vote": "BUY" | "SELL",
  "score": -1.0..1.0,
  "rationale": "3–6 sentences referencing the derived metrics or the data gap",
  "metrics": {{... or {{"error":"..."}} }}
}}
""",
    tools=[get_context_tool, fundamentals_web_tool, save_opinion_tool],
)

sent_agent = LlmAgent(
    name="sentiment",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=f"""
You are the Sentiment Agent.

YOUR ONLY JOB
- Use tools to:
  1) Fetch recent news for the ticker.
  2) Save a single sentiment opinion via save_opinion_tool.
- You MUST use tools. Do NOT answer with plain text.

TOOLS YOU CAN USE
- news_fetch_web_tool: fetch recent news for the given ticker.
- save_opinion_tool: save your final opinion. You MUST call this exactly once.

INPUT
You will receive a single JSON object with:
{{
  "analysis_id": "<string>",
  "ticker": "{TICKER}",
  "round": <1 or 2>,
  "lookback_days": <int>,
  "peer_views": [ ... ]   // optional
}}

You MUST:
- Read analysis_id, ticker, round, and lookback_days from this input.
- Use the `round` value from the input EXACTLY. Do NOT invent or change it.

REQUIRED BEHAVIOR (NO EXCEPTIONS)
1. FIRST, call news_fetch_web_tool with:
   {{
     "ticker": ticker_from_input,
     "lookback_days": lookback_days_from_input
   }}

2. THEN, based ONLY on:
   - the news_fetch_web_tool results, and
   - the input JSON
   construct a single Python dict called `payload` with ALL of these keys:

   {{
     "agent": "sentiment",              # must be exactly this string
     "ticker": <ticker_from_input>,     # e.g. "{TICKER}"
     "round": <round_from_input>,       # 1 or 2
     "vote": "BUY" or "SELL",           # no other values
     "score": <float between -1.0 and 1.0>,
     "rationale": <2-5 sentences>,
     "sample": [
       {{
         "title": <headline title>,
         "publisher": <publisher name>,
         "date": <YYYY-MM-DD>
       }},
       ...
     ]
   }}

   - If news_fetch_web_tool returns an error or zero items:
     - You MUST STILL choose BUY or SELL.
     - Use a very small absolute score (e.g. -0.1 or 0.1).
     - In the rationale, clearly mention that data was missing or limited.
     - You may set "sample" to an empty list [] in that case.

3. FINALLY, you MUST call save_opinion_tool EXACTLY ONCE with:
   {{
     "analysis_id": analysis_id_from_input,
     "agent_name": "sentiment",
     "payload": payload
   }}


STRICT CONSTRAINTS
- Do NOT print or return any natural-language explanation.
- Do NOT output the JSON payload directly to the user.
- The ONLY thing you should do is:
  - call news_fetch_web_tool once, then
  - call save_opinion_tool once.
- If you are unsure, you MUST STILL call save_opinion_tool with your best-guess payload.
""",
    tools=[get_context_tool, news_fetch_web_tool, save_opinion_tool],
)


minutes_agent = LlmAgent(
    name="minutes",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an equity research scribe.

GOAL
Turn the multi-agent equity debate into concise 'meeting minutes' for an investment committee.

STYLE
- Output plain Markdown (no JSON, no code fences).
- Be concise but complete.
- Assume the reader is a professional investor.

INPUT
You will receive a single JSON object with:
{
  "context": {
    "analysis_id": "...",
    "ticker": "...",
    "company": "...",
    "lookback_days": int,
    "rf_annual": float
  },
  "debate_log": [
    {
      "agent": "technical" | "fundamental" | "sentiment",
      "round": 1 | 2,
      "view": {
        "vote": "BUY" | "SELL",
        "score": float,
        "rationale": str,
        "metrics": {...}   // may be present for some agents
      }
    },
    ...
  ],
  "final_decision": "BUY" | "SELL"
}

WHAT TO PRODUCE
Write meeting minutes with the following structure:

1. Title line: "<TICKER> – Multi-Agent Debate Summary (Final: BUY/SELL)"
2. Section **Summary**: 2–4 bullet points with the main conclusion and key drivers.
3. Section **Agent Views**:
   - One bullet per agent, combining rounds:
     - Technical – stance, key metrics (return, momentum, vol, Sharpe), overall vote.
     - Fundamental – stance, key profitability / leverage metrics, overall vote.
     - Sentiment – tone of news/analyst commentary, overall vote (if any).
4. Section **Key Metrics**:
   - List only the most important metrics that were referenced (e.g. margins, fcf_margin, debt_to_equity, recent return, volatility, Sharpe).
5. Section **Risks / Watchpoints**:
   - 2–4 bullets with the main risks or uncertainties mentioned.
6. Section **Recommended Action**:
   - One sentence with the final recommended action (e.g. "Maintain SELL rating and avoid adding exposure near term.").

CONSTRAINTS
- Do NOT invent metrics; only use what appears in the debate_log views.
- If sentiment is missing, mention that explicitly ("No sentiment opinion was recorded; decision based on technical and fundamental only.").
- Keep total length to ~250–400 words.
""",
    tools=[],
)

## Deterministic Backstops for Missing Agent Opinions

This cell is optional. It defines three backstop functions that act as safety nets whenever an LLM agent fails to save an opinion for a given round. Instead of leaving gaps in the debate, these backstops generate a simple, rule-based BUY/SELL view using the same data sources as the agents. Together, these backstops ensure that every agent always has a vote in each round, even if the LLM fails to call tools or errors out, keeping the debate and consensus logic robust and fully populated.

In [10]:
def ensure_technical_backstop(round_id: int):
    if not last_opinion(ANALYSIS_ID, "technical"):
        m = prices_web_metrics_tool(TICKER, LOOKBACK_DAYS, RF_ANNUAL)
        if m.get("status") == "success":
            met = m["metrics"]
            buy_score = 0.0
            if (met.get("annualized_return", -1) > 0 and
                (met.get("momentum_21d", 0) > 0 or met.get("momentum_63d", 0) > 0) and
                met.get("sharpe", -1) > 0):
                vote, buy_score = "BUY", 0.15
            else:
                vote, buy_score = "SELL", -0.15
            save_opinion_tool(ANALYSIS_ID, "technical", {
                "agent":"technical","ticker":TICKER,"round":round_id,
                "vote": vote, "score": buy_score,
                "rationale":"Backstop: derived signal from annualized return, Sharpe and short-horizon momentum.",
                "metrics": met
            })
        else:
            save_opinion_tool(ANALYSIS_ID, "technical", {
                "agent":"technical","ticker":TICKER,"round":round_id,
                "vote":"SELL","score":-0.05,
                "rationale":"Backstop: price tool failed; issuing low-confidence SELL.",
                "metrics":{"error":m.get("error_message","fetch failure")}
            })

def ensure_fundamental_backstop(round_id: int):
    if not last_opinion(ANALYSIS_ID, "fundamental"):
        m = fundamentals_web_tool(TICKER)
        if m.get("status") == "success":
            met = m["metrics"]
            pos = ( (met.get("net_margin") or 0) > 0 and
                    (met.get("fcf_margin") or 0) > 0 and
                    (met.get("debt_to_equity") is None or met.get("debt_to_equity") < 2.5) )
            vote, score = ("BUY", 0.12) if pos else ("SELL", -0.12)
            save_opinion_tool(ANALYSIS_ID, "fundamental", {
                "agent":"fundamental","ticker":TICKER,"round":round_id,
                "vote": vote, "score": score,
                "rationale":"Backstop: derived from net/FCF margins and debt-to-equity threshold.",
                "metrics": met
            })
        else:
            save_opinion_tool(ANALYSIS_ID, "fundamental", {
                "agent":"fundamental","ticker":TICKER,"round":round_id,
                "vote":"SELL","score":-0.05,
                "rationale":"Backstop: fundamentals tool failed; issuing low-confidence SELL.",
                "metrics":{"error":m.get("error_message","fetch failure")}
            })

def ensure_sentiment_backstop(round_id: int):
    if not last_opinion(ANALYSIS_ID, "sentiment"):
        m = news_fetch_web_tool(TICKER, LOOKBACK_DAYS)
        if m.get("status") == "success" and m.get("items"):
            items = m["items"]
            # naive keyword scoring
            pos_kw = ("beat", "surge", "record", "upgrade", "raise", "positive", "strong", "outperform")
            neg_kw = ("miss", "fall", "recall", "downgrade", "cut", "negative", "weak", "investigation")
            score = 0
            for it in items:
                t = (it.get("title") or "").lower()
                score += sum(k in t for k in pos_kw)
                score -= sum(k in t for k in neg_kw)
            if score > 0:
                vote, conf = "BUY", min(0.1 + 0.02*score, 0.2)
            elif score < 0:
                vote, conf = "SELL", max(-0.1 - 0.02*abs(score), -0.2)
            else:
                vote, conf = "SELL", -0.05
            save_opinion_tool(ANALYSIS_ID, "sentiment", {
                "agent":"sentiment","ticker":TICKER,"round":round_id,
                "vote": vote, "score": conf,
                "rationale":"Backstop: keyword-skew from recent headlines.",
                "sample": items[:5]
            })
        else:
            save_opinion_tool(ANALYSIS_ID, "sentiment", {
                "agent":"sentiment","ticker":TICKER,"round":round_id,
                "vote":"SELL","score":-0.05,
                "rationale":"Backstop: no news returned; issuing low-confidence SELL.",
                "sample":[]
            })

## Orchestrating the Two-Round Multi-Agent Debate & Final Consensus

This cell runs the full debate loop for a single ticker and then summarizes the outcome:

**1. Context Initialization**

Builds ctx_payload with analysis_id, ticker, company, lookback_days, and rf_annual.

Passes this to intake_agent via InMemoryRunner which stores shared context using save_context_tool.

Defines a common base payload that each specialist agent (technical, fundamental, sentiment) will receive.

**2. Round 1 – Independent Analyses**

Calls run_agent for:

tech_agent

fund_agent

sent_agent

Each agent runs without peer views, forming its own initial BUY/SELL opinion.

After each call, last_opinion is checked; if an agent failed to save an opinion, a warning is printed and (optionally) a backstop can be invoked to fill the gap.

**3. Round 2 – Peer-Aware Debate**

For each agent, builds a payload that includes:

The same base context, round as 2 and all peer views that contains the other agents’ Round-1 opinions.

Each agent now sees its peers’ views and can reinforce or adjust its own stance based on the debate instructions in its prompt.

**4. Debate Transcript Printing**

Prints a readable transcript to gives a transparent view of how each agent reasoned and evolved its opinion across rounds.

**5. Final Consensus Rule**

consensus_decision collects the final votes from all agents and check the majority of BUY or SELL.

If meet a tie, then defer to the Technical Agent as the tie-breaker.

Finally, prints the final consensus decision for the ticker along with the rule used.

Overall, this cell ties together context, two-stage debate, logging, and a deterministic voting rule to produce a single, committee-style BUY/SELL decision from the three specialized LLM agents.

In [12]:
ctx_payload = {
    "analysis_id": ANALYSIS_ID,
    "ticker": TICKER,
    "company": COMPANY,
    "lookback_days": LOOKBACK_DAYS,
    "rf_annual": RF_ANNUAL,
}
await InMemoryRunner(intake_agent).run_debug(json.dumps(ctx_payload, indent=2))

base = {
    "analysis_id": ANALYSIS_ID,
    "ticker": TICKER,
    "company": COMPANY,
    "lookback_days": LOOKBACK_DAYS,
    "rf_annual": RF_ANNUAL,
    "peer_views": [],
}

# -------------------------
# Round 1 — independent analyses
# -------------------------

# TECHNICAL
await run_agent(tech_agent, {**base, "round": 1})
if not last_opinion(ANALYSIS_ID, "technical"):
    print("[WARN] technical agent did not save an opinion in round 1 → using backstop.")
    #ensure_technical_backstop(1)

# FUNDAMENTAL
await run_agent(fund_agent, {**base, "round": 1})
if not last_opinion(ANALYSIS_ID, "fundamental"):
    print("[WARN] fundamental agent did not save an opinion in round 1 → using backstop.")
    #ensure_fundamental_backstop(1)

# SENTIMENT
await run_agent(sent_agent, {**base, "round": 1})
if not last_opinion(ANALYSIS_ID, "sentiment"):
    print("[WARN] sentiment agent did not save an opinion in round 1 → using backstop.")
    #ensure_sentiment_backstop(1)

# -------------------------
# Round 2 — each agent reads peers and may revise vote
# -------------------------

# TECHNICAL
await run_agent(
    tech_agent,
    {**base, "round": 2, "peer_views": peer_snapshot(ANALYSIS_ID, "technical")}
)
if not last_opinion(ANALYSIS_ID, "technical"):
    print("[WARN] technical agent did not save a round-2 opinion → using backstop.")
    #ensure_technical_backstop(2)

# FUNDAMENTAL
await run_agent(
    fund_agent,
    {**base, "round": 2, "peer_views": peer_snapshot(ANALYSIS_ID, "fundamental")}
)
if not last_opinion(ANALYSIS_ID, "fundamental"):
    print("[WARN] fundamental agent did not save a round-2 opinion → using backstop.")
    #ensure_fundamental_backstop(2)

# SENTIMENT
await run_agent(
    sent_agent,
    {**base, "round": 2, "peer_views": peer_snapshot(ANALYSIS_ID, "sentiment")}
)
if not last_opinion(ANALYSIS_ID, "sentiment"):
    print("[WARN] sentiment agent did not save a round-2 opinion → using backstop.")
    #ensure_sentiment_backstop(2)

print("\n==================== DEBATE TRANSCRIPT ====================\n")
for evt in DEBATE_LOG.get(ANALYSIS_ID, []):
    agent = evt["agent"]; rnd = evt["round"]; view = evt["view"]
    print(f"({agent.upper()}, Round {rnd}) vote={view.get('vote')} score={view.get('score')}")
    print(f"— {view.get('rationale','')}\n")

print("===========================================================\n")
def consensus_decision(analysis_id: str):
    def _v(name): 
        op = last_opinion(analysis_id, name); 
        return op.get("vote") if op else None
    v_tech, v_fun, v_sen = _v("technical"), _v("fundamental"), _v("sentiment")
    votes = [v for v in (v_tech, v_fun, v_sen) if v in ("BUY","SELL")]
    if not votes:
        return "SELL"
    buys, sells = votes.count("BUY"), votes.count("SELL")
    if buys == sells:
        return v_tech or "SELL"
    return "BUY" if buys > sells else "SELL"
final_decision = consensus_decision(ANALYSIS_ID)
print(f"FINAL CONSENSUS for {TICKER}: **{final_decision}**")
print("Rule: majority after Round 2; tie-breaker = Technical Agent.")


 ### Created new session: debug_session_id

User > {
  "analysis_id": "tsla_equity_agent",
  "ticker": "TSLA",
  "company": "Tesla",
  "lookback_days": 30,
  "rf_annual": 0.04
}
intake > {"analysis_id": "tsla_equity_agent", "ticker": "TSLA", "company": "Tesla", "lookback_days": 30, "rf_annual": 0.04}


 ### Created new session: debug_session_id

User > {
  "analysis_id": "tsla_equity_agent",
  "ticker": "TSLA",
  "company": "Tesla",
  "lookback_days": 30,
  "rf_annual": 0.04,
  "peer_views": [],
  "round": 1
}
[DEBUG save_opinion_tool] SUCCESS for technical round 1

 ### Created new session: debug_session_id

User > {
  "analysis_id": "tsla_equity_agent",
  "ticker": "TSLA",
  "company": "Tesla",
  "lookback_days": 30,
  "rf_annual": 0.04,
  "peer_views": [],
  "round": 1
}
fundamental > The provided metrics for Tesla (TSLA) indicate a generally positive financial standing, though with some areas for consideration. The company exhibits a strong free cash flow margin of 14.2%, suggesti

## Generating Clean Meeting Minutes from the Debate

In short, this cell converts the structured multi-agent debate into a compact, investment-committee style note that users can read or paste directly into a research report.

In [16]:
minutes_input = {
    "context": CONTEXT.get(ANALYSIS_ID, {}),
    "debate_log": DEBATE_LOG.get(ANALYSIS_ID, []),
    "final_decision": final_decision,
}

async def run_agent(agent: LlmAgent, payload: Dict[str, Any]):
    runner = InMemoryRunner(agent)
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        result = await runner.run_debug(json.dumps(payload, indent=2))
    return result
meeting_minutes = await run_agent(minutes_agent, minutes_input)
meeting_minutes[0].content.parts[0]

Part(
  text="""TSLA – Multi-Agent Debate Summary (Final: BUY)

**Summary**
The investment committee has decided on a BUY recommendation for Tesla (TSLA), driven by positive fundamental metrics and optimistic sentiment. While technical indicators suggest short-term weakness, the company's strong free cash flow generation and a compelling narrative around AI and autonomous driving are seen as outweighing these concerns.

**Agent Views**
*   **Technical**: Maintained a SELL stance throughout the debate. Key metrics showed a negative annualized return (-16.7%) and Sharpe ratio (-0.20) over the last 30 days, with mixed momentum signals (positive 5-day, negative 21-day). High annualized volatility (49.28%) was also noted.
*   **Fundamental**: Recommended BUY, citing strong financial health. Key metrics included a robust FCF margin (14.2%), positive net income, and a low debt-to-equity ratio (0.17). Gross margin (18.0%) and operating margin (6.6%) were noted as areas for monitoring.
*   **Se